# Assignment

**Goal**: The goal of this assignment is to use the packages presented before, i.e. pydantic, pandera and optuna, in a real case scenario, where you want to train a model and find the best hyperparameters.

Using an open source insurance dataset, we will do the different steps:
* Validate the dataset, making sure it is prepared to train a model using pandera.
* Validate the input parameters for the bayesian optimization using pydantic.

**Table of contents**:
1. [Validate the dataset using pandera](#1-validate-the-dataset-using-pandera)
2. [Validate the parameters using pydantic](#2-validate-the-parameters-using-pydantic)
3. [Bonus](#3-bonus)

# Imports

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().parent))  # adjust .parent depth so 'src' is findable

## Packages

In [2]:
from typing import Any

import pandas as pd
import pandera.pandas as pa
from sklearn.metrics import root_mean_squared_error
from pydantic import (
    BaseModel,
    Field,
    StrictStr,
    field_validator,
)

In [3]:
from src.train_utils import (
    load_conf_parameters,
    retrieve_data_w_features,
    run_bayesian_optimization,
)

/Users/benjaminwallyn/Git/workshop-robust-hyperparam-tuning/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Options

In [4]:
DATA_PATH = "../data/01_raw"

In [5]:
pd.options.display.max_columns = 150

## Dataset

In [6]:
df = pd.read_parquet(f"{DATA_PATH}/fremotor1prem0304.parquet")

In [7]:
df.head(5)

,IDpol,Year,DrivAge,DrivGender,MaritalStatus,BonusMalus,LicenceNb,PayFreq,JobCode,VehAge,VehClass,VehPower,VehGas,VehUsage,Garage,Area,Region,Channel,Marketing,PremTot,test_set,val_set,big_train_set,train_set
0,1000111.100,2003.0,44.0,F,Cohabiting,50.0,3.0,Half-yearly,Private employee,10.0,Cheaper,P10,Regular,Private+trip to office,Closed zbox,A2,Headquarters,A,M1,144.1,1,0,0,0
1,1000113.100,2003.0,26.0,F,Cohabiting,85.0,2.0,Annual,Other,8.0,Cheapest,P8,Regular,Private+trip to office,Opened collective parking,A7,Headquarters,A,M2,215.3,1,0,0,0
2,1000113.100,2003.0,27.0,F,Cohabiting,106.0,2.0,Half-yearly,Other,6.0,Cheaper,P11,Regular,Private+trip to office,Opened collective parking,A7,Headquarters,A,M2,611.6,1,0,0,0
3,1000173.100,2003.0,52.0,M,Cohabiting,50.0,2.0,Half-yearly,Private employee,2.0,Cheaper,P11,Regular,Private+trip to office,Closed zbox,A7,Headquarters,A,M1,415.2,0,0,1,1
4,1000173.101,2003.0,52.0,M,Cohabiting,50.0,2.0,Half-yearly,Private employee,1.0,Cheap,P13,Regular,Private+trip to office,Closed collective parking,A7,Headquarters,A,M3,487.8,0,0,1,1


# 1. Validate the dataset using pandera

**Goal**: The goal of this section is to validate the dataframe we will use to train our model.

We want to validate the dataframe before training our model. Your goal is to make sure the columns will verify the following rules:
* **Year**: Check the year are between 2003 and 2004.
* **DrivAge**: Make sure the driver's age are possible (e.g. between 18 and 100).
* **DrivGender**: The gender is either 'M' or 'F'.
* **MaritalStatus**: Possible values "Cohabiting", "Married", "Single", "Widowed" or "Divorced".
* **BonusMalus**: The value of the bonus / malus is over 50.
* **LicenceNb**: The licence number is over 1.
* **JobCode**: The possible values are "Private employee", "Public employee", "Retiree", "Other", "Craftsman", "Farmer" or "Retailer",
* **VehAge**: Make sure the vehicule age is possible.
* **VehGas**: Either "Regular" or "Diesel".
* **Area**: Possible values are from A1 to A12 included.

**Exercise**: Update the schema to check the rules defined above.

In [8]:
######################
### YOUR CODE HERE ###
######################

However, as you can see, some features contain NaN values. To handle this issue, you have several solutions:
* In the dataframe schema from pandera, set the option of possible NaN values to True. We don't recommend this approach as many models can't handle NaN values or you want to make sure to use them properly.
* You can set the option in the pandera schema to drop rows with NaN values.
* You can handle it with the classical feature engineering technics (feature imputing, creating new category, etc.)

**Exercise**: Create a function to handle missing values.

<details>

<summary>Click to reveal tip</summary>

Use the `mode()[0]` function to get the most represented category.

In [9]:
######################
### YOUR CODE HERE ###
######################

To train the model, splitting the dataset between train, validation and test sets is important. You want to make sure each row is splitted into one and only one of the sets. To do so, you can use custom functions from pandera.

**Exercise**: Adapt the schema to check that each row is in one and only one set.
1. Add the right checks for the columns `train_set`, `val_set` and `test_set`.
2. Add a schema check to make sure that each row is assigned to one and only one set.

<details>

<summary>Click to reveal tip</summary>

Use the `checks` parameter of the schema directly.

In [10]:
######################
### YOUR CODE HERE ###
######################

We also have the `big_train_set` column which indicates rows contained in the train or validation sets.

**Exercise**: Adapt the schema to check that rows in either train or validation sets are in the big train set.

In [11]:
######################
### YOUR CODE HERE ###
######################

In [12]:
# Solution
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """Fill missing values.

    Fill the missing values in the 'JobCode' and 'MaritalStatus' columns with the mode
    (most frequent value) of their respective columns.
    
    Args:
        df (pd.DataFrame): Input DataFrame with potential missing values.
    """
    df["JobCode"] = df["JobCode"].fillna(df["JobCode"].mode()[0])
    df["MaritalStatus"] = df["MaritalStatus"].fillna(df["MaritalStatus"].mode()[0])
    return df

schema_df = pa.DataFrameSchema(
    {
        "IDpol": pa.Column(str),
        "PayFreq": pa.Column(str, checks=pa.Check.isin(["Annual", "Half-yearly", "Quarterly", "Monthly"])),
        "VehClass": pa.Column(str, checks=pa.Check.isin([
            "Cheapest", "Cheaper", "Cheap", "Medium low", "Medium", "Medium high", "Expensive", "More expensive", "Most expensive",
        ])),
        "VehPower": pa.Column(str, checks=pa.Check.isin([f"P{i}" for i in range(1, 21)])),
        "VehUsage": pa.Column(str, checks=pa.Check.isin([
            "Private+trip to office", "Professional", "Professional run",
        ])),
        "Garage": pa.Column(str, checks=pa.Check.isin([
            "Closed zbox", "Closed collective parking", "Opened collective parking", "Street",
        ])),
        ######################
        ### YOUR CODE HERE ###
        ######################
        "Year": pa.Column(int, checks=[pa.Check.ge(2003), pa.Check.le(2004)], coerce=True),
        "DrivAge": pa.Column(int, checks=[pa.Check.ge(18), pa.Check.le(100)], coerce=True),
        "DrivGender": pa.Column(str, checks=pa.Check.isin(["M", "F"])),
        "MaritalStatus": pa.Column(str, checks=pa.Check.isin([
            "Cohabiting", "Married", "Single", "Widowed", "Divorced",
        ])),
        "BonusMalus": pa.Column(int, checks=pa.Check.ge(50), coerce=True),
        "LicenceNb": pa.Column(int, checks=pa.Check.ge(1), coerce=True),
        "JobCode": pa.Column(str, checks=pa.Check.isin([
            "Private employee", "Public employee", "Retiree", "Other", "Craftsman", "Farmer", "Retailer", "Unknown",
        ])),
        "VehAge": pa.Column(int, checks=pa.Check.ge(0), coerce=True),
        "VehGas": pa.Column(str, checks=pa.Check.isin(["Regular", "Diesel"])),
        "Area": pa.Column(str, checks=pa.Check.isin([f"A{i}" for i in range(1, 13)])),
        # Sets for the model
        "train_set": pa.Column(int, checks=pa.Check.isin([0, 1])),
        "val_set": pa.Column(int, checks=pa.Check.isin([0, 1])),
        "test_set": pa.Column(int, checks=pa.Check.isin([0, 1])),
        "big_train_set": pa.Column(int, checks=pa.Check.isin([0, 1])),
    },
    checks=[
        pa.Check(
            lambda df: (df[["train_set", "val_set", "test_set"]].sum(axis=1) == 1).all(),
            error="Exactly one and only of the train_set, val_set and test_set must be 1 for each row.",
        ),
        pa.Check(
            lambda df: (
                (df["big_train_set"] == ((df["train_set"] == 1) | (df["val_set"] == 1)).astype(int))
            ).all(),
            error="big_train_set must be 1 if either train_set or valid_set is 1, and 0 otherwise."
        ),
    ]
)

df_filled = fill_missing_values(df)

df_validated = schema_df.validate(df_filled)

# 2. Validate the parameters using pydantic

**Goal**: The goal of this section is to display a real use of the pydantic validations to check that the parameters passed to your code are ones you accept.
The validations will be applied to parameters to use for the optuna modelisation.

**Exercise**: Create a pydantic class to validate the parameters given in a conf file to make sure they respect some rules to catch bugs before running your code:
1. Define the attribute `search_space` which represents the search space for the Bayesian optimization.
2. Define the attribute `categorical_feat` which represents the categorical features.
3. Define the attribute `default_params` which represents the default combination of hyperparameters to test at the beginning of the Bayesian optimization.
4. Create a custom validator to validate the `search_space`. The search space should be a dict where the keys are parameters names and the values are dict containing:
   * `sampling_type`: The type of the parameter. Should be `categorical`, `int` or `float`.
   * If the sampling type is categorical, an element `choices` containing the possible values.
   * If the sampling type is either int or float, elements `min` and `max` containing the minimum and maximum values for the search space.

For example:
```yaml
search_space: {
    "param_1": {
        "sampling_type": "categorical",
        "choices": ["cat_1", "cat_2"],
    },
    "param_2":{
        "sampling_type": "int",
        "min": min_val,
        "max": max_val,
    },
}
```

In [13]:
# Solution
class validate_input_parameters(BaseModel):
    """A pydantic class to validate the input parameters of the process."""
    target_name: StrictStr = Field(
        pattern=r"^[A-Za-z0-1\_]+$",
        description="Name of the target column.",
        frozen=True,
    )
    # ========================
    # ==== YOUR CODE HERE ====
    # ========================
    search_space: dict[str, dict[StrictStr, Any]] = Field(
        description="Search space for the Bayesian optimization.",
        frozen=True,
    )
    categorical_feat: list[StrictStr] = Field(
        description="List of the categorical features",
        frozen=True,
    )
    default_params: list[dict[StrictStr, Any]] = Field(
        description="List of combination of default parameters to check at the beginning of the bayesian optimization.",
        frozen=True,
    )

    @field_validator("search_space")
    @classmethod
    def validate_search_space(cls, value: dict[str, Any]) -> dict[str, Any]:
        """Validate the search space parameter for the bayesian optimization."""
        for hyparam_name, sampling_params in value.items():
            if sampling_params["sampling_type"] == "categorical":
                # For categorical parameters
                if "choices" not in sampling_params.keys():
                    raise KeyError(
                        f"Check {hyparam_name}. For a categorical feature, you should provide a 'choices' parameter for the possible values."
                    )
            elif sampling_params["sampling_type"] in ["float", "int"]:
                # For continuous parameters
                if ("min" not in sampling_params.keys()) | ("max" not in sampling_params.keys()):
                    raise KeyError(f"Check {hyparam_name}. For continuous parameters, you should provide 'min' and 'max' parameters.")
            else:
                raise ValueError(f"Check {hyparam_name}. Sampling type should be either 'categorical', 'int' or 'float'. You provided {sampling_params['sampling_type']}.")
        return value

In [14]:
params = load_conf_parameters("../conf/parameters.yml")

validated_params = validate_input_parameters(**params)

As you can see, some parameters don't follow the rules you defined in the pydantic class!

**Exercise**: Fix the parameters in the conf file to follow rules you defined.

In [15]:
#####################
### ACTIONS TO DO ###
#####################

Then, we can run the Bayesian optimization:

In [16]:
cols_to_drop = ["IDpol", "Year", "train_set", "val_set", "test_set", "big_train_set"]

X_big_train, y_big_train = retrieve_data_w_features(df=df, features_to_drop=cols_to_drop, split="big_train_set")
X_train, y_train = retrieve_data_w_features(df=df, features_to_drop=cols_to_drop, split="train_set")
X_val, y_val = retrieve_data_w_features(df=df, features_to_drop=cols_to_drop, split="val_set")
X_test, y_test = retrieve_data_w_features(df=df, features_to_drop=cols_to_drop, split="test_set")

best_params = run_bayesian_optimization(
    df_train=X_train,
    y_train=y_train,
    df_val=X_val,
    y_val=y_val,
    categorical_features=validated_params.categorical_feat,
    search_params=validated_params.search_space,
    default_params_list=validated_params.default_params,
)

best_params

/Users/benjaminwallyn/Git/workshop-robust-hyperparam-tuning/.venv/lib/python3.13/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-01 22:20:52,407] A new study created in memory with name: basic_hgb_opt
Best trial: 0. Best value: 107.462:   2%|▏         | 1/50 [00:00<00:25,  1.91it/s]

[I 2025-11-01 22:20:52,939] Trial 0 finished with value: 107.46195150088059 and parameters: {'max_iter': 50, 'learning_rate': 0.1, 'l2_regularization': 1.0}. Best is trial 0 with value: 107.46195150088059.


Best trial: 0. Best value: 107.462:   4%|▍         | 2/50 [00:00<00:17,  2.71it/s]

[I 2025-11-01 22:20:53,200] Trial 1 finished with value: 112.03645313813234 and parameters: {'max_iter': 44, 'learning_rate': 0.4758500101408589, 'l2_regularization': 7.319939418114051}. Best is trial 0 with value: 107.46195150088059.


Best trial: 0. Best value: 107.462:   8%|▊         | 4/50 [00:01<00:15,  2.90it/s]

...

Best trial: 48. Best value: 106.114:  98%|█████████▊| 49/50 [00:27<00:00,  1.34it/s]

[I 2025-11-01 22:21:19,533] Trial 48 finished with value: 106.11371766638698 and parameters: {'max_iter': 94, 'learning_rate': 0.14650511854697806, 'l2_regularization': 4.80666423314504}. Best is trial 48 with value: 106.11371766638698.


Best trial: 48. Best value: 106.114: 100%|██████████| 50/50 [00:27<00:00,  1.80it/s]

[I 2025-11-01 22:21:20,198] Trial 49 finished with value: 106.97829670481916 and parameters: {'max_iter': 96, 'learning_rate': 0.17971247269577406, 'l2_regularization': 5.583428202094666}. Best is trial 48 with value: 106.11371766638698.


{'max_iter': 94,
 'learning_rate': 0.14650511854697806,
 'l2_regularization': 4.80666423314504}

Now that we have found the best hyperparameters, we can train on the big train set and evaluate on the test set!

**Exercise**: Train a final model on the big train set using the best hyperparameters and evaluate on the test set.

In [17]:
# Solution
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(
    categorical_features=validated_params.categorical_feat,
    early_stopping=True,
    random_state=42,
    **best_params,
)
model.fit(X=X_big_train, y=y_big_train, X_val=X_test, y_val=y_test)

big_train_predictions = model.predict(X_big_train)
test_predictions = model.predict(X_test)

big_train_rmse = root_mean_squared_error(y_true=y_big_train, y_pred=big_train_predictions)
print(f"Big Train RMSE: {big_train_rmse}")
test_rmse = root_mean_squared_error(y_true=y_test, y_pred=test_predictions)
print(f"Test RMSE: {test_rmse}")

Big Train RMSE: 87.34474200996922
Test RMSE: 103.72454358053689


# 3. Bonus

You can see that there is overfitting! 

**Exercise**: Modify the function in the `train_utils.py` file to limit the overfitting of the model.

<details>

<summary>Click to reveal tip 1</summary>

Add an element in the metric to optimize containing the delta between train and validation.

<details>

<summary>Click to reveal tip 2</summary>

Use an alpha parameter to control the impact of this delta.

In [18]:
# Solution
import optuna
import numpy as np
from src.train_utils import build_search_space

def optimize_hyperparams_hgb(
    trial: optuna.trial.Trial,
    search_params: dict[str, Any],
    df_train: pd.DataFrame,
    y_train: pd.Series,
    df_val: pd.DataFrame,
    y_val: pd.Series,
    categorical_features: list[str],
    alpha: float = 0.3
) -> float:
    """Optimize the hyperparameters of the HistGradientBoosting model.
    
    Args:
        trial (optuna.trial.Trial): The Optuna trial object.
        search_params (dict[str, Any]): The search space parameters.
        df_train (pd.DataFrame): The training features.
        y_train (pd.Series): The training target.
        df_val (pd.DataFrame): The validation features.
        y_val (pd.Series): The validation target.
        categorical_features (list[str]): List of categorical feature names.

    Returns:
        (float): The validation loss.
    """
    # Build search space
    hyperparams = build_search_space(trial, search_params)

    # Define the model
    model = HistGradientBoostingRegressor(
        categorical_features=categorical_features,
        early_stopping=True,
        random_state=42,
        **hyperparams,
    )
    model.fit(X=df_train, y=y_train, X_val=df_val, y_val=y_val)
    val_predictions = model.predict(df_val)
    rmse_train = root_mean_squared_error(y_true=y_train, y_pred=model.predict(df_train))
    rmse_val = root_mean_squared_error(y_true=y_val, y_pred=val_predictions)

    return rmse_val + alpha * np.abs(rmse_train - rmse_val)